# 0. Import Packages&libraries

In [2]:
#!pip install tqdm

  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 6.8 MB/s eta 0:00:00


In [1]:
#import packages & libraries
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers
import pandas as pd
import os
import glob
from sklearn.model_selection import train_test_split, KFold,StratifiedKFold
from scipy.ndimage import rotate
from tqdm.keras import TqdmCallback
import numpy as np
import pydicom
from skimage import measure
import h5py
import re

2023-08-16 18:02:40.813623: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


## 0. GPU Check

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

2023-08-16 18:02:45.289083: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-08-16 18:02:45.291298: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-08-16 18:02:45.847504: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:04:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2023-08-16 18:02:45.851451: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:05:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2023-08-16 18:02:45.855438: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:08:0

In [3]:
import tensorflow as tf

# GPU 사용 가능 여부 확인
if tf.config.experimental.list_physical_devices('GPU'):
    print("GPU is available.")
else:
    print("GPU is not available.")

GPU is available.


In [16]:
!nvidia-smi

Mon Aug 14 23:15:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:04:00.0 Off |                  N/A |
| 23%   28C    P8     8W / 250W |   8971MiB / 11178MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:05:00.0 Off |                  N/A |
| 23%   

In [2]:
import torch

# GPU 사용 가능 여부 확인
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU 가속 사용 가능")
else:
    device = torch.device("cpu")
    print("GPU 가속 사용 불가능")

# 간단한 연산을 GPU에서 실행
a = torch.tensor([1.0, 2.0, 3.0], device=device)
b = torch.tensor([4.0, 5.0, 6.0], device=device)
c = a + b

print("연산 결과:", c)

GPU 가속 사용 가능
연산 결과: tensor([5., 7., 9.], device='cuda:0')


## 1. hdf5 file load, 정규화, 키값 추출

In [17]:
# 주 데이터 폴더 경로
main_data_folder = "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast"
sub_folder_names = ['001~100', '101~200', '201~300', '301~400', '401~514']

def create_hdf5_paths(folder_names, main_folder):
    hdf5_folders = [f"{main_folder}/{folder_name}/padded_hdf5_{folder_name.replace('~', '-')}" for folder_name in folder_names]
    return [f"{folder}/padded_{folder.split('/')[-2]}_data.h5" for folder in hdf5_folders]

def extract_info_from_hdf5(file_paths):
    all_patient_keys = []
    min_vals, max_vals = [], []

    for file_path in file_paths:
        with h5py.File(file_path, 'r') as f:
            all_patient_keys.extend(list(f.keys()))
            for key in f.keys():
                voxel = f[key][:]
                min_vals.append(np.min(voxel))
                max_vals.append(np.max(voxel))

    global_min = min(min_vals)
    global_max = max(max_vals)

    return all_patient_keys, global_min, global_max

file_paths = create_hdf5_paths(sub_folder_names, main_data_folder)
all_patient_keys, global_min, global_max = extract_info_from_hdf5(file_paths)

def load_hdf5_file(file_path, patient_num, min_val, max_val):
    with h5py.File(file_path, 'r') as f:
        voxel = f[str(patient_num)][:]
    voxel_normalized = (voxel - min_val) / (max_val - min_val) # 정규화
    return voxel_normalized

In [18]:
print(len(all_patient_keys))

512


In [7]:
'''
# 첫 번째 hdf5 파일 경로 가져오기
first_file_path = create_hdf5_paths(sub_folder_names, main_data_folder)[0]

# 첫 번째 hdf5 파일에서 첫 번째 환자의 데이터 가져오기
with h5py.File(first_file_path, 'r') as f:
    first_patient_key = list(f.keys())[0]
    first_patient_data = f[first_patient_key][:]
    
print(f"환자 번호: {first_patient_key}")
print(f"데이터 형태: {first_patient_data.shape}")
print(first_patient_data)
'''

환자 번호: 1
데이터 형태: (168, 286, 417)
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0.

In [8]:
'''
unique_values = np.unique(first_patient_data)
print(f"Unique values in the data: {unique_values}")
'''

Unique values in the data: [   0.           29.99462759   59.98925519   89.98388278  119.97851038
  149.97313797  179.96776557  209.96239316  239.95702076  269.95164835
  299.94627595  329.94090354  359.93553114  389.93015873  419.92478632
  449.91941392  479.91404151  509.90866911  539.9032967   569.8979243
  599.89255189  629.88717949  659.88180708  689.87643468  719.87106227
  749.86568987  779.86031746  809.85494505  839.84957265  869.84420024
  899.83882784  929.83345543  959.82808303  989.82271062 1019.81733822
 1049.81196581 1079.80659341 1109.801221   1139.7958486  1169.79047619
 1199.78510379 1229.77973138 1259.77435897 1289.76898657 1319.76361416
 1349.75824176 1379.75286935 1409.74749695 1439.74212454 1469.73675214
 1499.73137973 1529.72600733 1559.72063492 1589.71526252 1619.70989011
 1649.7045177  1679.6991453  1709.69377289 1739.68840049 1769.68302808
 1799.67765568 1829.67228327 1859.66691087 1889.66153846 1919.65616606
 1949.65079365 1979.64542125 2009.64004884 2039.634

In [5]:
'''
# 이전에 가져온 환자 번호와 라벨을 사용
labels = all_labels

# 환자 번호-라벨 매핑 딕셔너리 초기화
patient_label_map = dict(zip(all_patient_nums, all_labels))

# 파일 경로에서 환자 번호를 추출하고, 해당 번호에 대한 라벨을 찾아 저장
patient_numbers_from_files = []
filtered_labels = []

for file_path in file_paths:
    with h5py.File(file_path, 'r') as f:
        keys_in_file = sorted([int(key) for key in f.keys()])  # hdf5 파일 내부의 모든 키들을 정수로 변환 후 정렬
        for key in keys_in_file:
            patient_num = key  # 여기서는 이미 int 형태이므로 변환할 필요가 없습니다.
            patient_numbers_from_files.append(patient_num)
            
            # 해당 환자 번호의 라벨 찾기
            label = patient_label_map.get(patient_num)
            if label is not None:  # 해당 환자 번호에 대한 라벨이 있으면 추가
                filtered_labels.append(label)
 '''

In [7]:
'''
print(patient_numbers_from_files)
# patient_numbers_from_files의 개수 출력
print("Total patient numbers:", len(patient_numbers_from_files))

# 1부터 514 중 빠진 숫자 확인
missing_nums = set(range(1, 515)) - set(patient_numbers_from_files)
print("Missing patient numbers:", missing_nums)
'''

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

In [17]:
'''
print(len(patient_numbers_from_files))
print(len(labels))
print("First patient number:", patient_numbers_from_files[0])
print("Label for the first patient:", labels[0])

print("Second patient number:", patient_numbers_from_files[1])
print("Label for the second patient:", labels[1])

print("89th patient number:", patient_numbers_from_files[88])
print("Label for the 89th patient:", labels[88])
'''

512
512
First patient number: 1
Label for the first patient: 1
Second patient number: 2
Label for the second patient: 1
89th patient number: 89
Label for the 89th patient: 3


## 2. Data generator

In [19]:
def random_flip(image):
    flip_prob = np.random.uniform(0.0, 1.0)
    if flip_prob > 0.5:
        return np.fliplr(image)
    return image

def random_rotation(image):
    rotate_prob = np.random.uniform(0.0, 1.0)
    if rotate_prob > 0.5:
        angles = [90, 180, 270]
        angle = np.random.choice(angles)
        return rotate(image, angle, reshape=False)
    return image

def add_noise(image):
    noise_prob = np.random.uniform(0.0, 1.0)
    if noise_prob > 0.5:
        sigma = np.random.uniform(0.0, 0.5)
        noisy_image = np.clip(image + np.random.normal(0, sigma, image.shape), 0, 1)
        return noisy_image
    return image

def augment_data(image):
    image = random_flip(image)
    image = random_rotation(image)
    image = add_noise(image)
    return image

In [20]:
def data_generator(file_paths, batch_size, global_min, global_max):
    while True:
        batch_input = []
        for _ in range(batch_size):
            file_path = np.random.choice(file_paths)
            with h5py.File(file_path, 'r') as f:
                key = np.random.choice(list(f.keys()))
                input_data = f[key][:]
            input_data = (input_data - global_min) / (global_max - global_min) # 정규화
            augmented_data = augment_data(input_data) # 증강
            batch_input += [augmented_data]
        batch_x = np.array(batch_input)
        yield (batch_x, batch_x)

In [ ]:
'''
def get_hdf5_path_for_patient(patient_num, sub_folders, main_data_folder):
    for folder_name in sub_folders:
        start, end = map(int, folder_name.split('~'))
        if start <= patient_num <= end:
            return f"{main_data_folder}/{folder_name}/padded_hdf5_{folder_name.replace('~', '-')}/padded_{folder_name}_data.h5"
    return None
'''

## 3. Model structure

In [21]:
# multi-GPU 전략 설정
strategy = tf.distribute.MirroredStrategy()

print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')
Number of devices: 8


In [22]:
BATCH_SIZE_PER_REPLICA = 4
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [30]:
# strategy.scope() 내에서 모델을 정의
def autoencoder_structure():
    with strategy.scope():
        input_img = layers.Input(shape=(168, 286, 417, 1))  

        # Encoder
        x = layers.Conv3D(16, (3, 3, 3), activation='relu', padding='same')(input_img)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling3D((2, 2, 2), padding='same')(x)

        x = layers.MaxPooling3D((2, 2, 2), padding='same')(x)
        x = layers.Conv3D(32, (3, 3, 3), activation='relu', padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling3D((2, 2, 2), padding='same')(x)

        x = layers.Conv3D(32, (3, 3, 3), activation='relu', padding='same')(x)
        x = layers.MaxPooling3D((2, 2, 2), padding='same')(x)
        x = layers.Dropout(0.5)(x)

        x = layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same')(x)
        encoded = layers.MaxPooling3D((2, 2, 2), padding='same')(x)
        x = layers.Dropout(0.5)(encoded)

        # Decoder
        x = layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same')(x)
        x = layers.UpSampling3D((2, 2, 2))(x)
        x = layers.Dropout(0.5)(x)

        x = layers.Conv3D(32, (3, 3, 3), activation='relu', padding='same')(x)
        x = layers.UpSampling3D((2, 2, 2))(x)
        x = layers.Dropout(0.5)(x)

        x = layers.Conv3D(32, (3, 3, 3), activation='relu', padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.UpSampling3D((2, 2, 2))(x)

        x = layers.Conv3D(16, (3, 3, 3), activation='relu', padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.UpSampling3D((2, 2, 2))(x)

        decoded = layers.Conv3D(1, (3, 3, 3), activation='sigmoid', padding='same')(x)

        autoencoder = Model(input_img, decoded)
        autoencoder.compile(optimizer='adam', loss='mean_squared_error')
        
    return autoencoder 

## 4. Dataset Split(Dataloader)

In [31]:
# 데이터 분할 함수
def split_data(patient_keys, n_splits=5, test_size=0.3, random_state=42):
    patient_keys = np.array(patient_keys)
    
    train_valid_keys, test_keys = train_test_split(patient_keys, test_size=test_size, random_state=random_state)
    kf_train = KFold(n_splits=n_splits, random_state=random_state, shuffle=True)

    train_valid_splits = []

    for train_index, valid_index in kf_train.split(train_valid_keys):
        train_keys = train_valid_keys[train_index]
        valid_keys = train_valid_keys[valid_index]
        train_valid_splits.append((train_keys, valid_keys))

    return train_valid_splits, test_keys

def data_generator_from_keys(keys, file_paths, batch_size, global_min, global_max):
    while True:
        batch_input = []
        for _ in range(batch_size):
            key = np.random.choice(keys)
            for file_path in file_paths:
                with h5py.File(file_path, 'r') as f:
                    if key in f:
                        input_data = f[key][:]
                        break
            input_data = (input_data - global_min) / (global_max - global_min)  # 정규화
            augmented_data = augment_data(input_data)  # 증강
            batch_input.append(augmented_data)
        batch_x = np.array(batch_input)
        yield (batch_x, batch_x)

In [32]:
# 각 fold의 history를 저장할 리스트
all_histories = []

# 모델 저장 디렉토리 설정
MODEL_SAVE_DIR = "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/Save_breast"

# 모델 학습 함수에 사용할 generator도 key 기반으로 변경합니다.
def train_model(train_keys, valid_keys, fold_num):
    autoencoder = autoencoder_structure()  # 새로운 모델 인스턴스 생성
    train_generator = data_generator_from_keys(train_keys, file_paths, batch_size=GLOBAL_BATCH_SIZE, global_min=global_min, global_max=global_max)
    valid_generator = data_generator_from_keys(valid_keys, file_paths, batch_size=GLOBAL_BATCH_SIZE, global_min=global_min, global_max=global_max)
    
    history = autoencoder.fit(
        train_generator,
        epochs=30,
        shuffle=True,
        verbose=0,
        callbacks=[TqdmCallback()],
        steps_per_epoch=len(train_keys) * 10 // GLOBAL_BATCH_SIZE 
    )
    all_histories.append(history.history)

    # 모델 저장
    model_save_path = os.path.join(MODEL_SAVE_DIR, f"autoencoder_fold_{fold_num}.h5")
    autoencoder.save(model_save_path)

    return autoencoder

## 5. Training

In [33]:
train_valid_splits, test_keys = split_data(all_patient_keys)

for idx, (train_keys, valid_keys) in enumerate(train_valid_splits):
    autoencoder = train_model(train_keys, valid_keys, idx+1)

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

2023-08-16 21:32:20.572288: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
op: "FlatMapDataset"
input: "TensorDataset/_1"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_flat_map_fn_5080"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
  

INFO:tensorflow:batch_all_reduce: 26 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 26 all-reduces with algorithm = nccl, num_packs = 1


2023-08-16 21:33:45.240621: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_5_bfc) ran out of memory trying to allocate 4.78GiB (rounded to 5129220096)requested by op replica_5/model_1/conv3d_9/Conv3D
Current allocation summary follows.
2023-08-16 21:33:45.240622: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_2_bfc) ran out of memory trying to allocate 4.78GiB (rounded to 5129220096)requested by op replica_2/model_1/conv3d_9/Conv3D
Current allocation summary follows.
2023-08-16 21:33:45.240622: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_1_bfc) ran out of memory trying to allocate 4.78GiB (rounded to 5129220096)requested by op replica_1/model_1/conv3d_9/Conv3D
Current allocation summary follows.
2023-08-16 21:33:45.240630: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_6_bfc) ran out of memory trying to allocate 4.78GiB (rounded to 5129220096)requested by op replica_6/model_1/conv3d_9/Conv3D


ResourceExhaustedError: 9 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[4,16,168,286,417] and type float on /job:localhost/replica:0/task:0/device:GPU:1 by allocator GPU_1_bfc
	 [[node replica_1/model_1/conv3d_9/Conv3D (defined at /threading.py:980) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[4,16,168,286,417] and type float on /job:localhost/replica:0/task:0/device:GPU:6 by allocator GPU_6_bfc
	 [[node replica_6/model_1/conv3d_9/Conv3D (defined at /threading.py:980) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (2) Resource exhausted:  OOM when allocating tensor with shape[4,16,168,286,417] and type float on /job:localhost/replica:0/task:0/device:GPU:2 by allo

## 6. Encoder

In [ ]:
# 인코더 부분만을 포함하는 모델 생성
encoder = Model(input_img, encoded)

# 인코더 모델 저장
encoder_save_path = os.path.join(main_data_folder, "precon_encoder_model.h5")
encoder.save(encoder_save_path)

## 7. Visualization

In [ ]:
# 평균적인 성능 시각화
import numpy as np
import matplotlib.pyplot as plt

average_train_loss = [
    np.mean([x['loss'][i] for x in all_histories]) for i in range(30)
]
average_val_loss = [
    np.mean([x['val_loss'][i] for x in all_histories]) for i in range(30)
]

plt.plot(average_train_loss, label='Average Train Loss')
plt.plot(average_val_loss, label='Average Validation Loss')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
# 2. 인코더 특성 시각화
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer(name="encoded").output)

# 데이터의 일부분을 사용하여 임베딩을 생성
sample_data = np.array([load_hdf5_file(file) for file in file_paths[:30]])  # 처음 30개 데이터 사용
embeddings = encoder.predict(sample_data)

# 임베딩의 형태를 확인
print(embeddings.shape)

# PCA로 2D로 차원 축소
from sklearn.decomposition import PCA

embeddings_reshaped = embeddings.reshape(embeddings.shape[0], -1)  # 2D 배열로 변환
pca = PCA(n_components=2)
embeddings_pca = pca.fit_transform(embeddings_reshaped)

# 시각화
fig, axs = plt.subplots(10, 3, figsize=(15, 20))
for i in range(10):
    for j in range(3):
        idx = i * 3 + j
        axs[i, j].scatter(embeddings_pca[idx, 0], embeddings_pca[idx, 1], c=labels[idx], cmap='jet')
        axs[i, j].set_title(f"Sample {idx}")
plt.tight_layout()
plt.show()